# Text Correction - fastText 활용한 OCR 오인식 보안 
  
[목적]
- 주민등록증, 가족관계증명서 등 행정서류에서 관계(본인/자녀 등)를 추출하여 OCR 변환 시, 오인식으로 잘못된 관계정보 처리될 수 있음(e.g. 몬인, 지녀 등)
- 관계와 관련된 단어를 fastText 통해 학습하여 **사전에 등록된 단어로 변환**하여 정확도 향상

[활용 기술/라이브러리]
- fastText : 페이스북에서 word2vec(구글)의 단점을 보완하면서 만들어낸 알고리즘
- 문장 속 단어들의 조합으로 워드 임베딩을 하며, 이에 따라 학습에 사용된 적이 없는 단어에 대해서도 단어 벡터를 만들 수 있음
- 형태학적 측면의 유사성 판단 가능  
  &rarr; 우리는 **의미가 아닌 형태의 유사성으로만 판단하기 때문에 fastText 활용** 

[적용 내용]
- 모델 학습 위한 말뭉치의 경우, 단어+자모 형태로 구성하여 형태 유사성 판단 강화(자모 추가 여부에 따라 미세하게 차이남)
- 유사도 평가 시 단어-단어 비교 보다는 **자모분리한 단어와 자모분리한 단어를 비교**하면 원하는 결과 얻을 수 있음


## Import Packages

In [1]:
from gensim.models.fasttext import FastText
import gensim.models.word2vec
import numpy as np
from gensim import matutils
import re
import pandas as pd
from soynlp.hangle import decompose


In [2]:
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## fastText 모델
### 학습데이터  
[sample] 글자수에 따라 label 구분 임의 정의  
__label__1 부  
__label__2 본인  
__label__3 배우자  
__label__4 처고모부  
__label__6 ㄱ  
### 모델 생성 및 학습
- model = FastText(학습데이터셋, min_count, ....)
> min_count : 학습 시 지정한 숫자 보다 숫자보다 적게 발생하는 단어들은 학습하지 않음(빈도가 적은 단어들은 학습하지 않기 위함이나 우리는 문장 말뭉치가 아니여서 1로 지정)  
  word_ngrams : default word_ngrams=1 => n-gram 사용, 0 => 미사용(word2vec과 동일)  
  vector_size : 학습할 임베딩의 크기. 즉, 임베딩된 벡터의 차원  # 300이 적절  
  epchs : default epchs=5 &rarr; epoch 늘려도 성능 큰 차이 없음. 오히려 예상한 결과과 다른 단어가 우선수위화 됨  
  sg : default sg=0 => CBOW, if sg=1 => skip-gram (2가지 학습 알고리즘의 성능 차이는 크지 않음)  
  workers : 학습시 사용하는 프로세스 개수  
  window : context window 크기 # 5~10이 적절

In [3]:
corpus_path = 'relation_word.txt'
model_name = 'text_correciton_relation_model'

print('corpus 생성')
corpus = gensim.models.word2vec.Text8Corpus(corpus_path) 

print("학습")
model = FastText(corpus, min_count=1, word_ngrams=1, vector_size = 300, epochs=25)
model.save(model_name)

print(f"학습 소요 시간 : {model.total_train_time}")
print(model.wv.vectors.shape)

2022-09-01 16:35:12,198 : INFO : collecting all words and their counts
2022-09-01 16:35:12,199 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-09-01 16:35:12,200 : INFO : collected 146 word types from a corpus of 280 raw words and 1 sentences
2022-09-01 16:35:12,200 : INFO : Creating a fresh vocabulary
2022-09-01 16:35:12,201 : INFO : FastText lifecycle event {'msg': 'effective_min_count=1 retains 146 unique words (100.00% of original 146, drops 0)', 'datetime': '2022-09-01T16:35:12.201754', 'gensim': '4.2.0', 'python': '3.9.7 (default, Sep 16 2021, 08:50:36) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2022-09-01 16:35:12,202 : INFO : FastText lifecycle event {'msg': 'effective_min_count=1 leaves 280 word corpus (100.00% of original 280, drops 0)', 'datetime': '2022-09-01T16:35:12.202192', 'gensim': '4.2.0', 'python': '3.9.7 (default, Sep 16 2021, 08:50:36) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64

corpus 생성
학습


2022-09-01 16:35:14,957 : INFO : FastText lifecycle event {'update': False, 'trim_rule': 'None', 'datetime': '2022-09-01T16:35:14.957673', 'gensim': '4.2.0', 'python': '3.9.7 (default, Sep 16 2021, 08:50:36) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'build_vocab'}
2022-09-01 16:35:14,958 : INFO : FastText lifecycle event {'msg': 'training model with 3 workers on 146 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2022-09-01T16:35:14.958313', 'gensim': '4.2.0', 'python': '3.9.7 (default, Sep 16 2021, 08:50:36) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'train'}
2022-09-01 16:35:14,961 : INFO : EPOCH 0: training on 280 raw words (128 effective words) took 0.0s, 70276 effective words/s
2022-09-01 16:35:14,965 : INFO : EPOCH 1: training on 280 raw words (130 effective words) took 0.0s, 164022 effective words/s
2022-09-01 16:35:14,970 : INFO : EPOCH 2: training on 

학습 소요 시간 : 0.04544824899998545
(146, 300)


### 모델 선택


In [4]:
correction_model = FastText.load(model_name)

2022-09-01 16:35:18,404 : INFO : loading FastText object from text_correciton_relation_model
2022-09-01 16:35:18,407 : INFO : loading wv recursively from text_correciton_relation_model.wv.* with mmap=None
2022-09-01 16:35:18,408 : INFO : loading vectors_ngrams from text_correciton_relation_model.wv.vectors_ngrams.npy with mmap=None
2022-09-01 16:35:19,513 : INFO : setting ignored attribute buckets_word to None
2022-09-01 16:35:19,514 : INFO : setting ignored attribute vectors to None
2022-09-01 16:35:19,517 : INFO : setting ignored attribute cum_table to None
2022-09-01 16:35:19,519 : INFO : FastText lifecycle event {'fname': 'text_correciton_relation_model', 'datetime': '2022-09-01T16:35:19.519228', 'gensim': '4.2.0', 'python': '3.9.7 (default, Sep 16 2021, 08:50:36) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'loaded'}


## 공통 Function
- word_sim : 단어 유사도 측정  
- jamo_word : 자모 단위로 단어를 분해하여 유사도 측정하여 성능 향상

In [5]:
# 두 단어 리스트 사이의 유사도 측정
def word_sim(word_A, word_b, model=correction_model):
    return model.wv.n_similarity(word_A, word_b)


# 단어를 자모 단위로 분해
def jamo_word(sent):
    doublespace_pattern = re.compile('\s+')

    def transform(char):
        try:
            if char == ' ':
                return char
            jamo = decompose(char)
            if len(jamo) == 1:
                return jamo
            jamo_ = ''.join(c if c != ' ' else 'e' for c in jamo)
            return jamo_
        
        except Exception as e: # 마침표, 물음표 반환
            return char

    sent_ = ''.join(transform(char) for char in sent)
    sent_ = doublespace_pattern.sub(' ', sent_)
    return sent_
  


## 단어 사전과 비교하여 유사성 검사
- **jamo 분리해서 유사성 검증하면 정확도 향상**  
- 글자수 같은 것끼리만 판단하도록 적용  
    &rarr; 실제 오류 케이스 확인하여 1~2글자는 글자수 같을 때만 체크하고 나머지는 전체 비교할지 등 보완

In [6]:
word = '몬인'
dictionary = ['본인', '배우자', '부', '모', '자녀', '며느리', '사위', '시부', '시모', '장인', '장모', '백부', '백모', '숙부', '숙모', '양부', '양모', '형', '형수', '제', '제수', '시숙', '올케', '종형', '종형수', '종제', '종제수', '누이', '형부', '매', '매부', '시누이', '종매', '종매부', '제부', '조카', '동서', '손', '손부', '손서', '증손', '증손부', '고손', '고손부', '고종', '오빠', '배우자의자', '조부', '조모', '시조부', '시조모', '증조부', '증조모', '고조부', '고조모', '장조부', '장조모', '고모', '고모부', '대고모', '당숙', '당숙모', '서모', '서조모', '이질', '당질', '질부', '외숙', '외숙모', '외종', '이모', '이모부', '이종', '외손', '외손부', '외조부', '외조모', '처형', '처제', '처남', '처남댁', '처조카', '처질부', '처숙부', '처숙모', '처사촌', '처고모', '처고모부', '처고종', '처이모', '처이모부', '처이종', '처가친척', '시가친척', '외가친척', '친척', '동거인', '외증손', '외증손부', '언니', '시외조부', '시외조모', '누나', '계부', '계모']
results = []

for i in range(len(dictionary)):
    
    if len(word) == len(dictionary[i]):
        result_info = {}
        result_info['relation'] = dictionary[i]
        result_info['score'] = word_sim(jamo_word(word), jamo_word(dictionary[i]), correction_model) 
        #result_info['score'] = word_sim(dictionary[i], word, saved_model) 
        results.append(result_info)
        
results = sorted(results, key=lambda d:d['score'], reverse=True)
df = pd.DataFrame(results)
print('입력 단어:', word)
df.head(3)

입력 단어: 몬인


,relation,score
0,본인,0.999105
1,증손,0.997497
2,장인,0.997158
